<a href="https://colab.research.google.com/github/rodjfg/los-fabulosos-pixelotls/blob/master/Example_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install --quiet git+https://github.com/rodjfg/los-fabulosos-pixelotls

In [11]:
from los_fabulosos_pixelotls.tools import load_raw_data, select_by_areas, select_by_contrast, select_trials, calculate_mean_firing_rate
import numpy as np
from matplotlib import rcParams 
from matplotlib import pyplot as plt

rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True



In [12]:
# get data
alldat = load_raw_data()

# name conventions
regions = ["vis ctx", "thal", "hipp", "other ctx", "midbrain", "basal ganglia", "cortical subplate", "other"]
region_colors = ['blue', 'red', 'green', 'darkblue', 'violet', 'lightblue', 'orange', 'gray']
brain_groups = [
    ["VISa", "VISam", "VISl", "VISp", "VISpm", "VISrl"], # visual cortex
    ["CL", "LD", "LGd", "LH", "LP", "MD", "MG", "PO", "POL", "PT", "RT", "SPF", "TH", "VAL", "VPL", "VPM"], # thalamus
    ["CA", "CA1", "CA2", "CA3", "DG", "SUB", "POST"], # hippocampal
    ["ACA", "AUD", "COA", "DP", "ILA", "MOp", "MOs", "OLF", "ORB", "ORBm", "PIR", "PL", "SSp", "SSs", "RSP","TT"], # non-visual cortex
    ["APN", "IC", "MB", "MRN", "NB", "PAG", "RN", "SCs", "SCm", "SCig", "SCsg", "ZI"], # midbrain
    ["ACB", "CP", "GPe", "LS", "LSc", "LSr", "MS", "OT", "SNr", "SI"], # basal ganglia 
    ["BLA", "BMA", "EP", "EPd", "MEA"] # cortical subplate
    ]



In [14]:
gaussfilter = True
gauss_sigma = 1
selected_regions = ["VISp"]

all_d_right = []
all_d_left = [] 
all_preference_rates = [] # mean response to right stim - mean response to left stim
all_preference_components_rates = [] # mean response to right stim, mean response to left stim
Correct_examples = [] # Append corresponding results from different conditions to this list
Incorrect_examples = [] # Append corresponding results from different conditions to this list

for dat in alldat:
    # get important variables from specific experiment
    dt = dat['bin_size']
    response = dat['response'] # correct responses to right stim, nogo, left stim: -1, 0, 1
    vis_right = dat['contrast_right']
    vis_left = dat['contrast_left']
    N_neurons, N_trials, N_timebins  = dat['spks'].shape

    # find neurons indices from selected area
    neurons_indices_in_area = select_by_areas(dat, selected_regions = selected_regions)

    # selected trials per contrast level and response
    right_stim_correct_trials = select_trials(dat, contrast_pair=(0,1), response_type='to_left')
    right_stim_incorrect_trials   = select_trials(dat,contrast_pair=(0,1), response_type='to_right')

    #left_stim_correct_trials = select_trials(dat,contrast_pair=(1,0), response_type='to_right')
    #left_stim_wrong_trials   = select_trials(dat,contrast_pair=(1,0), response_type='to_left')

    #right_stim_trials = select_by_contrast(dat, contrast_pair=(0,1))
    #left_stim_trials  = select_by_contrast(dat, contrast_pair=(1,0))

    # find mean firing rates across trials for Right Stimulus
    correct_right_trace   = calculate_mean_firing_rate(dat['spks'][neurons_indices_in_area][:,right_stim_correct_trials], dt, ['trials'], gaussfilter = gaussfilter, gauss_sigma = gauss_sigma)
    incorrect_right_trace = calculate_mean_firing_rate(dat['spks'][neurons_indices_in_area][:,right_stim_incorrect_trials], dt, ['trials'], gaussfilter = gaussfilter, gauss_sigma = gauss_sigma)
    
    Correct_examples.append(correct_right_trace)
    Incorrect_examples.append(incorrect_right_trace)


